## Objective

In this experiment, we will test the validity of existing appointment scheduling heuristics by evaluating several instances of scheduling systems and analyzing the results. We will examine the performance of different scheduling strategies and compare them to the heuristics proposed in the literature. The goal is to find quasi optimal schedules that form a good starting point for further optimization using for instance local search algorithms.

## Background

The extensive body of research on appointment scheduling offers valuable heuristics for designing schedules that are likely close to optimal. In their seminal work, Welch et al. (1952)[^appointment-scheduling-heuristics-t-larger-n-1] established the following rule: schedule two patients at the beginning of the day to minimize the risk of idle time if one patient does not show up. The remaining patients should then be scheduled at intervals equal to the mean consultation time.

[^appointment-scheduling-heuristics-t-larger-n-1]: Welch, J. D., & Bailey, N. T. J. (1952). Appointment systems in hospital outpatient departments. The Lancet, 259(6718), 1105-1108.

!['Dome' shapes of appointment schedules. From Robinson and Chen (2003)[^appointment-scheduling-heuristics-t-larger-n-2]](images/robinson-chen-dome-shape.png)

[^appointment-scheduling-heuristics-t-larger-n-2]: Robinson, L. W., & Chen, R. R. (2003). Scheduling doctors' appointments: optimal and empirically-based heuristic policies. IIE Transactions, 35(3), 295-307.

[^appointment-scheduling-heuristics-t-larger-n-3]: Robinson, L. W., & Chen, R. R. (2003). Scheduling doctors' appointments: optimal and empirically-based heuristic policies. IIE Transactions, 35(3), 295-307.

Robinson and Chen (2003)[^appointment-scheduling-heuristics-t-larger-n-4] explored the structure of optimal schedules. In their model, the length of intervals could be adjusted to accommodate different job allowances. Their findings include the following general observations:

[^appointment-scheduling-heuristics-t-larger-n-4]: Robinson, L. W., & Chen, R. R. (2003). Scheduling doctors' appointments: optimal and empirically-based heuristic policies. IIE Transactions, 35(3), 295-307.

-   Job allowances follow a 'dome' pattern, with more time allotted to patients in the middle of the day.
-   The first job allowance is consistently much lower than the subsequent ones, while the final job allowance is also somewhat lower than the others.
-   Intermediate job allowances are generally uniform.

Klassen and Yoogalingam (2009)[^appointment-scheduling-heuristics-t-larger-n-5] observed a plateau form when appointment intervals are restricted to integers.

[^appointment-scheduling-heuristics-t-larger-n-5]: Klassen, K. J., & Yoogalingam, R. (2009). Improving performance in outpatient appointment services with a simulation optimization approach. Production and Operations Management, 18(4), 447-458.

In our model, all interval times are fixed integers, but job allowances can be adjusted by overbooking or underbooking selected service intervals. We examined the extent to which the above rules apply to our model definition.

## Methodology

### Tools and Materials


In [ ]:
import itertools
from joblib import Parallel, delayed
import networkx as nx
import plotly.graph_objects as go
import numpy as np
from typing import List, Tuple
import plotly.subplots as sp
from plotly.subplots import make_subplots
from concurrent.futures import ThreadPoolExecutor
import time
import math

### Experimental Design

We will compare the performance of the Bailey-Welch heuristic with the performance of a schedule generated by a local search algorithm. The local search algorithm will start from the Bailey-Welch schedule and iteratively improve it by swapping patients between time slots to reduce the patients' waiting time and physician's overtime.

### Variables

-   **Independent Variables**:
    -   Different schedule designs with increasingly large instances (lengths and number of patients).
-   **Dependent Variables**:
    -   Objective function value (weighted sum of average waiting time and physician's overtime) for each schedule.
    -   Computation times

### Samples

We will generate several schedules with different lengths ($T$), numbers of patients ($N$) and weights for patients' waiting time ($w$) relatively to overtime.

$$
N \in \{16, \dots , 22\}
$$ $$
T \in \{15, \dots , 20\}
$$ $$
w \in \{0.1, 0,9\}
$$ This means that the total solution space varies between `{python} int(math.comb(16+12-1, 16)/1e6)` mln and `{python} int(math.comb(22+20-1, 22)/1e9)` bln schedules. The challenge is to find the optimal schedules within these vast solution spaces.

### Experimental Procedure

We will evaluate multiple schedules using the Bailey-Welch heuristic and a local search algorithm, complemented by visualizations to compare their structures. Additionally, we will analyze computation times to assess the practical feasibility of the local search algorithm for large-scale instances.

## Results

We began with a set of relatively small instances, keeping the number of intervals fixed while varying the number of patients. For each instance, we applied the local search algorithm and compared the resulting optimized schedules to the initial ones. The initial schedules, generated using the Bailey-Welch heuristic, are depicted as blue dotted lines, while the optimized schedules obtained through local search are shown as red solid lines.


In [ ]:
from functions import create_random_schedules, calculate_objective, compute_convolutions, local_search, get_v_star, powerset, get_neighborhood, build_welch_bailey_schedule, build_quasi_optimal_schedule, local_search_w_intermediates

# Parameters
N = range(15, 19)
T = 20
s = [0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.2]  # Example service time probability distribution
d = 2  # Duration threshold
q = 0.1  # No-show probability
w = 0.01  # Weight for waiting time in the objective
v_star = get_v_star(T)

# Lists to store results
x_stars = []
x_initials = []  # To store initial schedules
obj_vals = []
schedules_list, objectives_list = [], []

# Iterate over each n in N
start = time.time()
for n in N:
    print(f'Running local search for schedule with N={n}')
    x = build_quasi_optimal_schedule(n, T)
    x_initials.append(x)  # Store the initial schedule
    convolutions = compute_convolutions(s, n, q)
    schedules, objectives = local_search_w_intermediates(x, d, convolutions, w, v_star, T)
    #x_star, obj = local_search(x, d, q, convolutions, w, v_star, T)
    obj_vals.append(objectives[-1])
    x_stars.append(schedules[-1])
    schedules_list.append(schedules)
    objectives_list.append(objectives)
end = time.time()
print("Optimized schedules:", x_stars)
print("Objective values:", obj_vals)
print(f"Search time: {end - start:.2f} seconds")

In [ ]:
# Number of subplots needed
num_subplots = len(x_stars)

# Create a subplot figure with one chart per subplot
fig = sp.make_subplots(
    rows=num_subplots, 
    cols=1, 
    shared_xaxes=True, 
    subplot_titles=[f'n = {n}' for n in N]
)

# Add each initial and optimized schedule to its respective subplot
for idx, (x_initial, x_star) in enumerate(zip(x_initials, x_stars)):
    # Add initial schedule as a dotted line
    fig.add_trace(
        go.Scatter(
            x=list(range(T)), 
            y=x_initial, 
            mode='lines', 
            name='Initial schedule' if idx == 0 else None,  # Show legend only once
            line=dict(dash='dot', color='blue')
        ), 
        row=idx + 1, 
        col=1
    )
    
    # Add optimized schedule as a solid line with markers
    fig.add_trace(
        go.Scatter(
            x=list(range(T)), 
            y=x_star, 
            mode='lines+markers', 
            name='Optimized schedule' if idx == 0 else None,  # Show legend only once
            line=dict(color='red')
        ), 
        row=idx + 1, 
        col=1
    )

# Update layout properties
fig.update_layout(
    height=600 * num_subplots,  # Adjust height based on the number of subplots
    title=dict(
        text=f"Optimal schedules across different values of N\n(T={T}, w={w})",
        x=0.5,  # Center the title horizontally
        # y=0.95,  # Adjust the vertical position (closer to the top)
        font=dict(size=20),  # Optional: Adjust title font size
        pad=dict(b=50)  # Add padding at the top of the title
    ),
    xaxis_title="Time slot (x)",
    yaxis_title="# of patients (y)",
    template="plotly_white",
    showlegend=False  # Enable legend to distinguish between initial and optimized schedules
)

# Set consistent y-axis ticks for each subplot
for i in range(1, num_subplots + 1):
    fig.update_yaxes(tickmode='linear', tick0=0, dtick=1, row=i, col=1)

# Optionally, adjust the legend position
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

# Show the Plotly figure
fig.show()

Subsequently we've analyzed the number of steps required for the local search algorithm to transition from the initial schedule to the optimal schedule, as well as the characteristics of the search paths. To accomplish this, we've recorded and visualized the search paths using graph representations for multiple instances starting at the initial schedule and ending at the optimized schedule. Each node in the graph represents an improved schedule. The length of the edges corresponds to the improvement in the objective function value.

::: column-page

In [ ]:
for idx, (n, schedules, objectives) in enumerate(zip(N, schedules_list, objectives_list), start=1):
    print(f'Processing N={n}, number of schedules: {len(schedules)}')
    
    # Create individual network graph
    individual_fig = create_schedule_network_from_lists(
        schedules=schedules,
        objective_values=objectives,
        echo=False
    )
    
    individual_fig.update_layout(
        autosize=False,
        width=1500,
        height=1800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        )
    )
    
    # Show the individual network graph
    individual_fig.show()

:::

## Discussion

The results of the experiments appear to confirm that optimal schedules adhere to the Bailey-Welch rule of scheduling two patients at the start of the day. The remaining patients are then allocated as evenly as possible across the remaining time intervals. When patients' time is valued higher than the physician's, empty intervals are inserted into the schedule to absorb potential spillover times. This adjustment, however, pushes some patients out of the schedule, thereby increasing the total overtime.

The local search algorithm required on average 22 and 42 steps for both types of small instances ($w = 0.1$ and $w = 0.9$, respectively) to compute the global optimum. Interestingly the number of steps required to reach the optimal schedule did not increase significantly for the larger instance ($T=20$, $N=\{21, 22\}$, $w = 0.5$), despite the larger solution space. This suggests that the local search algorithm is effective in finding the optimal schedule within a reasonable number of steps.

In each instance, the most significant reductions in the objective value occurred each time a patient was moved from the last time interval to an earlier one. This adjustment minimized overtime, which apparently has a substantial impact on the objective value.

## Timeline

This experiment was started at 18-11-2024 and is expected to be completed by 31-12-2024.

## References